In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import ipywidgets as widgets
from IPython.display import display

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
from db.models import TickerReturn, Ticker
from db.configuration import database_connection

In [4]:
import pendulum
start_period = pendulum.naive(2019, 10, 1)

In [5]:
tickers_list = ["T.TO", "BB", "ATD-B.TO", "REAL.TO", "QTRH.TO", "DRM.TO", "MX.TO", "KEY.TO", "FTT.TO", "BAM-A.TO", "BYD.TO", 
                "L", "ADSK", "FSLY", "ANET", "ASML", "BERY", "BWA", "SCHW", "FLGT", "WAL", "TWTR", "SHOP"]
exlist = tickers_list

In [6]:
from trade.configuration import Configuration
exlist = Configuration().scrape().tickers

In [7]:
tickers = Ticker.select().where(Ticker.ticker.in_(exlist))
base_query = TickerReturn.select(
    TickerReturn.datetime,
    TickerReturn.close,
).where(
    TickerReturn.interval == '1d',
    TickerReturn.datetime > start_period
).order_by(
    TickerReturn.datetime.asc()
).join(Ticker)

In [8]:
dataframes = []
for ticker in exlist:
    ticker_query = base_query.where(TickerReturn.ticker == Ticker.get(Ticker.ticker == ticker))
    dataframe = pd.read_sql(ticker_query.sql()[0], database_connection(),
        params=ticker_query.sql()[1],
        index_col='datetime'
    )
    df = pd.DataFrame({ticker: dataframe.close}, index=dataframe.index)
    print(ticker)
    print(df.shape)
    dataframes.append(df)
dataframes = pd.concat(dataframes, axis=1) #.dropna()

T.TO
(324, 1)
BB
(324, 1)
ATD-B.TO
(323, 1)
REAL.TO
(324, 1)
QTRH.TO
(324, 1)
DRM.TO
(324, 1)
MX.TO
(324, 1)
KEY.TO
(324, 1)
FTT.TO
(324, 1)
BAM-A.TO
(324, 1)
BYD.TO
(324, 1)
L
(324, 1)
ADSK
(324, 1)
FSLY
(324, 1)
ANET
(324, 1)
ASML
(324, 1)
BERY
(324, 1)
BWA
(324, 1)
SCHW
(324, 1)
FLGT
(324, 1)
WAL
(324, 1)
TWTR
(324, 1)
SHOP
(324, 1)
ALS.TO
(324, 1)
NWH-UN.TO
(13, 1)
WSO
(324, 1)
FB
(12, 1)
SQ
(324, 1)
UPWK
(12, 1)
SWKS
(12, 1)
AAPL
(324, 1)
AMZN
(12, 1)


In [9]:
ticker_returns = dataframes.pct_change().dropna()

In [10]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [11]:
mu = expected_returns.return_model(dataframes, method="capm_return")
S = risk_models.risk_matrix(dataframes, method="oracle_approximating")

In [12]:
ef = EfficientFrontier(mu, S)

In [13]:
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [14]:
cleaned_weights

OrderedDict([('T.TO', 0.00868),
             ('BB', 0.00198),
             ('ATD-B.TO', 0.00662),
             ('REAL.TO', 0.0),
             ('QTRH.TO', 0.00183),
             ('DRM.TO', 0.0),
             ('MX.TO', 0.01447),
             ('KEY.TO', 0.00027),
             ('FTT.TO', 0.0),
             ('BAM-A.TO', 0.00703),
             ('BYD.TO', 0.00037),
             ('L', 0.00261),
             ('ADSK', 0.0),
             ('FSLY', 0.00413),
             ('ANET', 0.00799),
             ('ASML', 0.0),
             ('BERY', 0.00675),
             ('BWA', 0.01023),
             ('SCHW', 0.0),
             ('FLGT', 0.0037),
             ('WAL', 0.00254),
             ('TWTR', 0.0),
             ('SHOP', 0.0),
             ('ALS.TO', 0.0),
             ('NWH-UN.TO', 0.07056),
             ('WSO', 0.0016),
             ('FB', 0.18739),
             ('SQ', 0.01382),
             ('UPWK', 0.32006),
             ('SWKS', 0.19899),
             ('AAPL', 0.0),
             ('AMZN', 0.12835)])

In [15]:
ef.portfolio_performance(verbose=True)

Expected annual return: 49.0%
Annual volatility: 9.2%
Sharpe Ratio: 5.09


(0.49008951866822065, 0.09233637443316621, 5.091054544365667)

In [16]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(cleaned_weights, dataframes.iloc[-1], total_portfolio_value=10000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
pd.Series(alloc).sort_index()

Leftover: $17.62


ANET          1
ATD-B.TO      2
BAM-A.TO      2
BB            2
BERY          2
BWA           3
FB            8
FLGT          1
FSLY          1
KEY.TO        1
L             1
MX.TO         4
NWH-UN.TO    54
QTRH.TO       7
SQ            1
SWKS         11
T.TO          4
UPWK         68
WAL           1
WSO           1
dtype: int64